In [27]:
%%capture
!python -m pip install onnx onnxruntime
!python -m pip install abraia

import os
if not os.getenv('ABRAIA_ID') and not os.getenv('ABRAIA_KEY'):
    #@markdown <a href="https://abraia.me/console/gallery" target="_blank">Upload and manage your images</a>
    abraia_id = ''  #@param {type: "string"}
    abraia_key = ''  #@param {type: "string"}
    %env ABRAIA_ID=$abraia_id
    %env ABRAIA_KEY=$abraia_key

from abraia import Abraia

multiple = Abraia()

In [28]:
# import torch
# import torchvision

# dummy_input = torch.randn(1, 3, 224, 224)
# model = torchvision.models.mobilenet_v2(pretrained=True)
# model.eval()

# torch.onnx.export(model, dummy_input, "model.onnx", verbose=True, input_names=['input'], output_names=['output'])

# multiple.upload_file("model.onnx", "camera/model.onnx")

In [29]:
import onnx
import torch
from abraia.torch import load_classes, load_model, export_onnx

dataset = 'hymenoptera_data'

class_names = load_classes(os.path.join(dataset, 'model_ft.txt'))
model = load_model(os.path.join(dataset, 'model_ft.pt'), class_names)
model_path = export_onnx(os.path.join(dataset, 'model_ft.onnx'), model)
# model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


device cpu


In [30]:
import numpy as np
import onnxruntime as ort
from PIL import Image


ort_session = ort.InferenceSession(f"/tmp/{model_path}", providers=['CPUExecutionProvider'])


def resize(img, size):
    width = size if img.height > img.width else round(size * img.width / img.height)
    height = round(size * img.height / img.width) if img.height > img.width else size
    return img.resize((width, height))


def crop(img, size):
    left, top = (img.width - size) // 2, (img.height - size) // 2
    right, bottom = left + size, top + size
    return img.crop((left, top, right, bottom))


def normalize(img, mean, std):
    img = (np.array(img) / 255. - np.array(mean)) / np.array(std)
    return img.astype(np.float32)


def preprocess(img):
    img = resize(img, 256)
    img = crop(img, 224)
    img = normalize(img, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    return np.expand_dims(img.transpose((2, 0, 1)), axis=0)


def predict(src):
    img = Image.open(src)
    input = preprocess(img)
    outputs = ort_session.run(None, {"input": input})
    a = np.argsort(-outputs[0].flatten())
    results = {}
    for i in a[0:5]:
        results[class_names[i]]=float(outputs[0][0][i])
    return results


filename = 'dog.jpg'
multiple.download_file(os.path.join(dataset, filename), filename)
predict(filename)

{'dogs': 2.679403066635132,
 'bees': -0.37143513560295105,
 'ants': -1.542833685874939,
 'cats': -1.6038011312484741}